In [7]:
import json

# Load the annotation file
with open(r"C:\Users\moham\Desktop\computer vision\Axial-Dataset.v4-axial.coco\annotations\test.json", "r") as f:
    data = json.load(f)

# Filter out negative annotations
data['annotations'] = [ann for ann in data['annotations'] if ann['category_id'] != 2]  # Assuming '2' is the id for negative

# Remove the negative class from categories
data['categories'] = [cat for cat in data['categories'] if cat['name'] != 'negative']

# Save the updated annotations
with open(r"C:\Users\moham\Desktop\computer vision\Axial-Dataset.v4-axial.coco\annotations\test.json", "w") as f:
    json.dump(data, f)


In [10]:
from pycocotools.coco import COCO

# Path to your COCO annotation file
annotation_file = r"C:\Users\moham\Desktop\computer vision\Axial-Dataset.v4-axial.coco\annotations\train.json"

# Load the annotation file
coco = COCO(annotation_file)

# Get all the categories (classes)
categories = coco.loadCats(coco.getCatIds())
class_names = [cat['name'] for cat in categories]
print("Classes:", class_names)

# Get information about the images and annotations
image_ids = coco.getImgIds()
images = coco.loadImgs(image_ids)
print(f"Number of images: {len(images)}")

# Get annotations for a specific image
img_id = image_ids[0]  # Select the first image as an example
annotation_ids = coco.getAnnIds(imgIds=[img_id])
annotations = coco.loadAnns(annotation_ids)

print("Annotations for the first image:", annotations)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Classes: ['tumor', 'positive']
Number of images: 518
Annotations for the first image: []


In [2]:
import os
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import detectron2

In [11]:
register_coco_instances("mri_train", {}, r"C:\Users\moham\Desktop\computer vision\Axial-Dataset.v4-axial.coco\annotations\train.json",  r"C:\Users\moham\Desktop\computer vision\Axial-Dataset.v4-axial.coco\train")
register_coco_instances("mri_test", {}, r"C:\Users\moham\Desktop\computer vision\Axial-Dataset.v4-axial.coco\annotations\test.json",  r"C:\Users\moham\Desktop\computer vision\Axial-Dataset.v4-axial.coco\test")


In [12]:
from detectron2.data import DatasetCatalog, MetadataCatalog

# Unregister existing datasets if they are already registered
for d in ["mri_train", "mri_test"]:
    if d in DatasetCatalog.list():
        DatasetCatalog.remove(d)
    if d in MetadataCatalog.list():
        MetadataCatalog.remove(d)

# Now re-register
register_coco_instances("mri_train", {}, r"C:\Users\moham\Desktop\computer vision\Axial-Dataset.v4-axial.coco\annotations\train.json",  r"C:\Users\moham\Desktop\computer vision\Axial-Dataset.v4-axial.coco\train")
register_coco_instances("mri_test", {}, r"C:\Users\moham\Desktop\computer vision\Axial-Dataset.v4-axial.coco\annotations\test.json",  r"C:\Users\moham\Desktop\computer vision\Axial-Dataset.v4-axial.coco\test")


In [16]:
# Configuring the Fast R-CNN model
cfg = get_cfg()
cfg.merge_from_file(detectron2.model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("mri_train",)
cfg.DATASETS.TEST = ("mri_test",)  # For evaluation
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = detectron2.model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001  # Learning rate
cfg.SOLVER.MAX_ITER = 500  # Adjust based on your needs
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # Tumor, positive

# Setup output directory
cfg.OUTPUT_DIR = "./output_mri"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=False)

# Trainer setup
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

evaluator = COCOEvaluator("mri_test", cfg, False, output_dir=cfg.OUTPUT_DIR)
test_loader = build_detection_test_loader(cfg, "mri_test")  # No need to call it

[10/30 13:40:09 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (3, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (3,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (8, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[10/30 13:40:10 d2.engine.train_loop]: Starting training from iteration 0
[10/30 13:46:28 d2.utils.events]:  eta: 2:26:10  iter: 19  total_loss: 1.408  loss_cls: 1.004  loss_box_reg: 0.349  loss_rpn_cls: 0.02933  loss_rpn_loc: 0.004153    time: 19.2218  last_time: 34.0262  data_time: 0.5616  last_data_time: 0.0074   lr: 3.8962e-05  max_mem: 2174M
[10/30 13:52:26 d2.utils.events]:  eta: 2:15:46  iter: 39  total_loss: 1.009  loss_cls: 0.4976  loss_box_reg: 0.4114  loss_rpn_cls: 0.03844  loss_rpn_loc: 0.005148    time: 18.5349  last_time: 17.8460  data_time: 0.0056  last_data_time: 0.0069   lr: 7.8922e-05  max_mem: 2174M
[10/30 13:55:44 d2.utils.events]:  eta: 1:45:58  iter: 59  total_loss: 0.9422  loss_cls: 0.388  loss_box_reg: 0.5082  loss_rpn_cls: 0.02257  loss_rpn_loc: 0.005791    time: 15.5565  last_time: 2.5164  data_time: 0.0050  last_data_time: 0.0034   lr: 0.00011888  max_mem: 2174M
[10/30 13:57:48 d2.utils.events]:  eta: 1:06:27  iter: 79  total_loss: 0.9506  loss_cls: 0.3244  l

In [17]:
import random
import cv2
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

# Load the trained model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set a threshold for making predictions
predictor = detectron2.engine.DefaultPredictor(cfg)

# Load the test dataset
dataset_dicts = DatasetCatalog.get("mri_test")

# Visualize predictions on a few random test images
for d in random.sample(dataset_dicts, 3):  # Display predictions on 3 random images
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)  # Run inference

    # Visualize the predictions
    v = Visualizer(img[:, :, ::-1],
                   metadata=MetadataCatalog.get("mri_test"), 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW  # Remove the color of the unsegmented parts
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    # Display the image with predictions
    cv2.imshow("Predicted Image", v.get_image()[:, :, ::-1])
    cv2.waitKey(0)


[10/30 14:09:58 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output_mri\model_final.pth ...


c:\Users\moham\anaconda3\envs\detectron_env\lib\site-packages\fvcore\common\checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_locatio

WARNING [10/30 14:09:58 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[10/30 14:09:58 d2.data.datasets.coco]: Loaded 36 images in COCO format from C:\Users\moham\Desktop\computer vision\Axial-Dataset.v4-axial.coco\annotations\test.json
